In [1]:
%%cpp -a

//gallery includes
#include "common/gallery_includes.h"

//larsoft object includes
#include "common/larsoftobj_includes.h"

//setup our input file and input tag
std::vector<std::string> filenames = 
{
    "/pnfs/uboone/mc/uboone/reconstructed/prod_v06_26_07/prodgenie_bnb_intrinsic_nue_cosmic_uboone_MCC8Tune3_reco/reco/prodgenie_bnb_intrinsic_nue_cosmic_uboone_996_20180207T201546_gen2_d63adf8e-d13a-455f-aef5-fb1518a37df7_20180212T065449_reco1_20180212T081208_reco2.root"
};

art::InputTag vtx_tag = { "pandoraNu" };
art::InputTag pfp_tag = { "pandoraNu" };
art::InputTag gen_tag = { "generator" };

//geometry
float X_LENGTH=256.0;
float Y_HALF_HEIGHT=116.5;
float Z_LENGTH=1037.0;

void PrintDaughters(size_t i_pfp,
                    std::vector<recob::PFParticle> const&,
                    int ntab=0);

void PrintDaughters(size_t i_pfp,
                    std::vector<recob::PFParticle> const& pfp_vec,
                    int ntab)
{
    //std::cout << "\tN_daughters=" << pfp_vec[i_pfp].NumDaughters() << std::endl;
    for(size_t i_d=0; i_d<pfp_vec[i_pfp].NumDaughters(); ++i_d){
        for(int it=0; it<ntab; ++it) std::cout << "\t";
        std::cout << "Found daughter particle id=" << pfp_vec[i_pfp].Daughters()[i_d] << std::endl;
        PrintDaughters(pfp_vec[i_pfp].Daughters()[i_d],pfp_vec,ntab+1);
    }
    return;
}

void Run(){

for(gallery::Event ev(filenames) ; !ev.atEnd(); ev.next()) {
/*
    std::cout << "Processing " 
              << "Run " << ev.eventAuxiliary().run() << ", "
              << "Event " << ev.eventAuxiliary().event() << endl;
*/  
    
    auto const& mctruth_handle = ev.getValidHandle<std::vector<simb::MCTruth>>(gen_tag);
    auto const& truth_vec(*mctruth_handle);

    if(truth_vec.size()==0){
        std::cout << "ERROR! truth_vec size is " << truth_vec.size() << std::endl;
        break;
    }

    auto const& nu = truth_vec[0].GetNeutrino();
    std::cout << "Neutrino position is (" 
              << nu.Nu().Vx() << ","
              << nu.Nu().Vy() << ","
              << nu.Nu().Vz() << ")"
              << std::endl;
       
    if(nu.Nu().Vx()<0.0+10.0 || nu.Nu().Vx()>X_LENGTH-10.0 ||
       nu.Nu().Vx()<-1*Y_HALF_HEIGHT+20.0 || nu.Nu().Vx()>Y_HALF_HEIGHT-20.0 ||
       nu.Nu().Vz()<0.0 || nu.Nu().Vz()>Z_LENGTH-10.0
       )
        continue;

    
    auto const& pfp_handle = ev.getValidHandle<std::vector<recob::PFParticle>>(pfp_tag);
    auto const& pfp_vec(*pfp_handle);

    for (auto const& pfp : pfp_vec){
        if(!pfp.IsPrimary()) continue;
        std::cout << "Found primary particle id=" << pfp.Self() << std::endl;
        PrintDaughters(pfp.Self(),pfp_vec,1);
    }
    
    /*
    auto const& vtx_handle = ev.getValidHandle<std::vector<recob::Vertex>>(vtx_tag);
    auto const& vtx_vec(*vtx_handle);
    std::cout << "There are " << vtx_vec.size() << " vertices in this event." << std::endl;
        
    for(auto const& vtx : vtx_vec){
        auto const& pos = vtx.position();
        std::cout << "\tPosition (x,y,z)="
                  << "(" << pos.X() << ","
                  << "" << pos.Y() << ","
                  << "" << pos.Z() << ")"
                  << std::endl;
    }
    */
    
    //auto const& pfp_vtx_handle = ev.getValidHandle<art::Assns<recob::PFParticle,recob::Vertex,void> >(pfp_tag);
    //auto const& pfp_vtx_assn_vec(*pfp_vtx_handle);
    //std::cout << "Assns: " << pfp_vtx_assn_vec.size();

    art::FindOne<recob::Vertex> vtx_per_pfp(pfp_handle,ev,pfp_tag);
    for(size_t i_pfp=0; i_pfp<pfp_vec.size(); ++i_pfp)
    {
        
        std::vector<recob::Vertex const*> vtx_ptrs;
        //vtx_per_pfp.get(i_pfp,vtx_ptrs);
        /*
        if(vtx_ptrs.size()==1){
            auto vtx(*(vtx_ptrs[0]));
            std::cout << "For pfparticle " << i_pfp << " found vertex at "
                  << "(" << vtx.position().X() << ","
                  << ""  << vtx.position().Y() << ","
                  << ""  << vtx.position().Z() << ")"
                  << std::endl;
                
        }
        */
            
    }
    
}//end loop over events

}

Info in <TUnixSystem::ACLiC>: creating shared library /uboone/app/users/wketchum/notebooks/Sample_Notebooks/75f4a9f0_C.so
In file included from /uboone/app/users/wketchum/notebooks/Sample_Notebooks/75f4a9f0_C_ACLiC_dict.cxx:40:0:
/uboone/app/users/wketchum/notebooks/Sample_Notebooks/75f4a9f0.C: In function ‘void PrintDaughters(size_t, const std::vector<recob::PFParticle>&, int)’:
/uboone/app/users/wketchum/notebooks/Sample_Notebooks/75f4a9f0.C:32:26: warning: comparison between signed and unsigned integer expressions [-Wsign-compare]
     for(size_t i_d=0; i_d<pfp_vec[i_pfp].NumDaughters(); ++i_d){
                       ~~~^~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


In [2]:
Run()

Warning in <TClass::Init>: no dictionary for class art::DoNotRecordParents is available


Successfully opened file /pnfs/uboone/mc/uboone/reconstructed/prod_v06_26_07/prodgenie_bnb_intrinsic_nue_cosmic_uboone_MCC8Tune3_reco/reco/prodgenie_bnb_intrinsic_nue_cosmic_uboone_996_20180207T201546_gen2_d63adf8e-d13a-455f-aef5-fb1518a37df7_20180212T065449_reco1_20180212T081208_reco2.root


Error in cling::AutoloadingVisitor::InsertIntoAutoloadingState:
   Missing FileEntry for GeoConstants.h
   requested to autoload type larlite::geo::View_t
Error in cling::AutoloadingVisitor::InsertIntoAutoloadingState:
   Missing FileEntry for GeoConstants.h
   requested to autoload type larlite::geo::SigType_t
Error in cling::AutoloadingVisitor::InsertIntoAutoloadingState:
   Missing FileEntry for GeoTypes.h
   requested to autoload type larlite::geo::TPCID
Error in cling::AutoloadingVisitor::InsertIntoAutoloadingState:
   Missing FileEntry for GeoTypes.h
   requested to autoload type larlite::geo::PlaneID
Error in cling::AutoloadingVisitor::InsertIntoAutoloadingState:
   Missing FileEntry for GeoTypes.h
   requested to autoload type larlite::geo::WireID
Error in cling::AutoloadingVisitor::InsertIntoAutoloadingState:
   Missing FileEntry for larlite_base.h
   requested to autoload type larlite::Message
Error in cling::AutoloadingVisitor::InsertIntoAutoloadingState:
   Missing FileEntr

Neutrino position is (270.839,0.552882,64.3579)
Neutrino position is (81.464,-40.8471,1110.51)
Neutrino position is (205.364,13.5053,-59.5439)
Neutrino position is (276.332,-20.5455,587.384)
Neutrino position is (20.0937,-139.725,130.462)
Found primary particle id=3
	Found daughter particle id=0
Found primary particle id=4
	Found daughter particle id=1
Found primary particle id=5
	Found daughter particle id=2
Neutrino position is (55.8618,36.0213,373.634)
Found primary particle id=8
	Found daughter particle id=0
Found primary particle id=9
	Found daughter particle id=1
Found primary particle id=10
	Found daughter particle id=3
Found primary particle id=11
	Found daughter particle id=2
Found primary particle id=12
	Found daughter particle id=4
	Found daughter particle id=6
	Found daughter particle id=7
Found primary particle id=13
	Found daughter particle id=5
Neutrino position is (273.224,-46.1353,909.431)
Neutrino position is (-8.94803,-132.463,445.982)
Neutrino position is (133.615,3